# Similarity Compared
----

In this notebook, we will compute vertex similarity scores using the various cuGraph algorithms.  We will then compare the similarities scores in tables.

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware        |
| --------------|------------|------------------|-----------------|-----------------------|
| Don Acosta    | 09/25/2023 | created          | 23.10 nightly   | AMPERE A6000 CUDA 11.7|


**Note: On large graphs these algorithms can take prohibitive time or memory. The notebook will show how to run on defined pairs instead.**

The Similarity algorithms in cuGraph use different methods to compare pairs of vertices. All of them use the intersection of the set of adjacent nodes for the set overlap. However each of the three algorithms differ on the denominator to determine the similarity coefficients. All three are normalized between zero and one. where zero is no overlap at all and one means identical adjacencies.

__Jaccard Similarity__<br>
The [Jaccard similarity](https://en.wikipedia.org/wiki/Jaccard_index) measure was developed by botonist, Paul Jaccard who used the measure to compare plant species. His work popularized the measure's use in in other fields as well.

It can be expressed as:<br>
$\text{Jaccard similarity} = \frac{|A \cap B|}{|A \cup B|}$

__Overlap Similarity__<br>
The [Overlap Similarity](https://en.wikipedia.org/wiki/Overlap_coefficient) is also known as the Szymkiewicz–Simpson coefficient. It is often used to compare binary and categorical data in the fields of Genome analysis, recommender systems and anomaly detection. It differs from the Jaccard measure above in that it uses the size of the smaller of the two set sizes as the denominator.

It can be expressed as

$oc(A,B)=\frac{|A|\cap|B|}{min(|A|,|B|)}$

__Sørensen-Dice Coefficient__<br>
The [Sørensen coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient#) is known as the Sørensen-Dice coefficient. It was independently developed for use by botonists Lee Raymond Dice and  Thorvald Sørensen. Although originating in the field of Botony, the coefficient is now used in computer vision, Natural Language Processing(NLP) and Data Mining among other fields.
It differs from Jaccard and Overlap in that the calculation doubles the intersection size and divides it by the sum of the two set sizes.

It can be expressed as

Sørensen coefficient = $\left(2 * |A \cap B| \right) \over \left(|A| + |B| \right)$



---
# Now for the code !

Load the required dependencies.

In [ ]:
import cugraph
from cugraph.datasets import dining_prefs
# only needed to display results in a table 
from IPython.display import display_html 

Function that calls all the cuGraph similarity/link prediction algorithms 

In [ ]:
def compute_similarity(G,pairs=None):
    _jdf = cugraph.jaccard(G,pairs)
    _jdf2 = _jdf[ (_jdf['first'] != _jdf['second'] ) ]
    _odf = cugraph.overlap(G,pairs)
    _odf2 = _odf[ (_odf['first'] != _odf['second'] ) ]
    _sdf = cugraph.sorensen_coefficient(G,pairs)
    _sdf2 = _sdf[ (_sdf['first'] != _sdf['second'] ) ]
    return _jdf2, _odf2, _sdf2

Function to put all the results in a convenient table

In [ ]:
# Print function
def print_similarity(jdf,odf,sdf,num_records=5):

    js_top = jdf.sort_values(by='jaccard_coeff', ascending=False).head(num_records).to_pandas()
    os_top = odf.sort_values(by='overlap_coeff', ascending=False).head(num_records).to_pandas()
    ss_top = sdf.sort_values(by='sorensen_coeff', ascending=False).head(num_records).to_pandas()
    
    df1_styler = js_top.style.set_table_attributes("style='display:inline'").set_caption('Jaccard').hide(axis='index')
    df2_styler = os_top.style.set_table_attributes("style='display:inline'").set_caption('Overlap').hide(axis='index')
    df3_styler = ss_top.style.set_table_attributes("style='display:inline'").set_caption('Sørensen').hide(axis='index')

    display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

Create the graph from the Dining preferences data set.

In [ ]:
G = dining_prefs.get_graph(download=True)

Run the three similarity Algorithms and print out the five links with the highest scores.

In [ ]:
jdf, odf, sdf = compute_similarity(G)
print_similarity(jdf,odf,sdf)

Now find the the complete set of two-hop neigbors and compare them instead of just using the existing one-hop edges. In a larger graph, this will run considerably faster since the default 

In [ ]:
# this cugraph algorithm pulls a set containing every pair of vertices
# that are within 2-hops of each other
two_hops_pairs = G.get_two_hop_neighbors()

jdf_hops, odf_hops, sdf_hops = compute_similarity(G,pairs=two_hops_pairs)
print_similarity(jdf_hops,odf_hops,sdf_hops)

### It's that easy with cuGraph

----
Copyright (c) 2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.